In [933]:

import requests
import warnings
from tqdm import tqdm, trange
import streamlit as st
import pandas as pd 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import unittest
import tables
import pickle
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.io as pio
import math
import investpy
import sklearn.preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import subprocess
import numpy as np
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [909]:
pio.templates.default = "plotly_dark"

In [1003]:
oi = yf.Ticker("XLK")

In [1044]:
sector_proxies = ['XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY']
sector_proxies_names = ['Materials', 'Communications', 'Energy', 'Financials', 'Industrials', 'Technology', 'Consumer Staples', 'Real Estate', 'Utilities', 'Healthcare', 'Consumer Discretionary']

In [1046]:
len(sector_proxies)

11

In [1038]:
hm = yf.Tickers(sector_proxies)

In [1069]:
yf.download(tickers=['XLE'],start = '2020-02-08',)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-10,53.700001,53.830002,53.380001,53.709999,43.077812,14640600
2020-02-11,54.439999,54.520000,54.070000,54.310001,43.559055,13844600
2020-02-12,55.119999,55.349998,54.590000,55.040001,44.144543,17199800
2020-02-13,54.889999,55.180000,54.549999,54.869999,44.008183,11974900
2020-02-14,55.060001,55.099998,54.240002,54.580002,43.775593,12520000
...,...,...,...,...,...,...
2025-02-03,87.629997,88.550003,86.830002,88.190002,88.190002,19908200
2025-02-04,87.779999,90.230003,87.629997,90.040001,90.040001,20139900
2025-02-05,90.010002,90.360001,89.510002,90.230003,90.230003,10886000


In [1070]:
df = hm.history(period='5y', interval='1d',auto_adjust=False)['Close']
df

[*********************100%%**********************]  11 of 11 completed


Ticker,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2020-02-10,60.060001,56.189999,53.709999,31.010000,84.120003,101.040001,64.529999,40.400002,68.849998,103.589996,129.759995
2020-02-11,60.389999,56.130001,54.310001,31.120001,84.440002,100.699997,64.290001,40.889999,69.099998,104.209999,130.779999
2020-02-12,60.720001,56.860001,55.040001,31.139999,85.230003,101.790001,64.260002,41.150002,69.150002,104.339996,132.119995
2020-02-13,60.500000,56.869999,54.869999,31.129999,84.750000,101.519997,64.639999,41.400002,69.919998,103.849998,131.889999
2020-02-14,60.509998,56.939999,54.580002,31.170000,84.639999,101.959999,64.790001,41.880001,70.389999,103.930000,131.690002
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-03,88.879997,102.599998,88.190002,51.259998,136.979996,227.699997,79.290001,41.349998,78.300003,147.440002,229.100006
2025-02-04,89.139999,103.519997,90.040001,51.139999,137.020004,230.979996,78.620003,41.349998,77.610001,147.039993,231.800003
2025-02-05,89.070000,102.519997,90.230003,51.680000,137.119995,234.190002,79.110001,42.000000,78.389999,148.449997,229.169998


In [1072]:
df.iloc[-1000,:]

Ticker
XLB      74.160004
XLC      73.559998
XLE      46.470001
XLF      32.220001
XLI      89.959999
XLK     136.919998
XLP      65.809998
XLRE     37.990002
XLU      61.970001
XLV     116.540001
XLY     170.559998
Name: 2021-02-17 00:00:00, dtype: float64

In [1064]:
lookback_period = 2
performance_df = ((df.iloc[-1,:]-df.iloc[-lookback_period,:])/df.iloc[-lookback_period,:])*100
performance_df = pd.DataFrame(performance_df, columns=["Value", ])
performance_df["Name"] = sector_proxies_names
performance_df 

,Value,Name
Ticker,,
XLB,-1.219508,Materials
XLC,-0.213676,Communications
XLE,-0.011293,Energy
XLF,-0.556517,Financials
XLI,-0.297665,Industrials
XLK,-0.855936,Technology
XLP,-0.151004,Consumer Staples
XLRE,-0.356120,Real Estate
XLU,-0.204347,Utilities


In [1063]:
temp = performance_df.sort_values(['Value'])
temp

,Value,Name
Ticker,,
XLY,-0.019558,Consumer Discretionary
XLB,-0.012195,Materials
XLK,-0.008559,Technology
XLF,-0.005565,Financials
XLV,-0.004690,Healthcare
XLRE,-0.003561,Real Estate
XLI,-0.002977,Industrials
XLC,-0.002137,Communications
XLU,-0.002043,Utilities


In [ ]:
performance_fig = go.Figure()
temp = performance_df.sort_values(['Value'])
performance_fig.add_trace(go.Bar(x = temp[0], y = temp["Name"], orientation='h'))
performance_fig.update_xaxes(title_text = f"Performance over {lookback_period-1} Days")
performance_fig.update_yaxes(title_text = "Sector")
performance_fig

In [ ]:
prices = pd.DataFrame(oi.history()['Close'])
prices

,Close
Date,
2025-01-08 00:00:00-05:00,233.880005
2025-01-10 00:00:00-05:00,228.779999
2025-01-13 00:00:00-05:00,227.160004
2025-01-14 00:00:00-05:00,227.759995
2025-01-15 00:00:00-05:00,232.289993
2025-01-16 00:00:00-05:00,230.500000
2025-01-17 00:00:00-05:00,234.110001
2025-01-21 00:00:00-05:00,236.059998
2025-01-22 00:00:00-05:00,241.389999
